In [1]:
import os
import cv2
import pickle 
import numpy as np
from tqdm import tqdm
from collections import Counter
from classRecogLib.faceRecog import loadDetectionModel,loadRecognitionModel,encodeFace,findFaces
from sklearn.neighbors import KNeighborsClassifier

2023-04-17 16:01:49.663128: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 16:01:50.856658: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/martin/anaconda3/envs/classLib/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-17 16:01:50.856792: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/martin/anaconda3/envs/classLib/lib/python3

In [2]:
modelR = loadRecognitionModel('models/')
modelD = loadDetectionModel()

In [3]:
encodings =[ ]
names = []
path= './moddedPics/'
for i in tqdm(os.listdir(path)):
    studentP = path+i+'/'
    for j in os.listdir(studentP):
        img = cv2.imread(studentP+j)
        img = cv2.resize(img,(160,160))
        enco = encodeFace(img,modelR)
        encodings.append(enco)
        names.append(i)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
100%|██████████| 26/26 [00:32<00:00,  1.26s/it]


In [4]:
model = KNeighborsClassifier(n_neighbors=6,weights='distance',algorithm='ball_tree')
model.fit(encodings,names)

KNeighborsClassifier(algorithm='ball_tree', n_neighbors=6, weights='distance')

In [ ]:
with open('knnPickleFile.pickle', 'wb') as f:
    pickle.dump(model, f)
with open("namesFile.pickle", "wb") as f:   
    pickle.dump(names, f)

In [5]:
def knn_weighted_vote(distances, predicted_classes, threshold):
    """
    Takes in the distances and predicted classes from a k-nearest neighbors model, groups the predicted
    classes by frequency, weights the frequency by distance, and returns the predicted class if the
    highest weighted frequency is above a given threshold.
    
    Args:
        distances (numpy.ndarray): Array of distances.
        predicted_classes (list): List of predicted classes.
        threshold (float): Threshold for the highest weighted frequency.
        
    Returns:
        str: Predicted class if the highest weighted frequency is above the given threshold, otherwise None.
    """
    # Find the unique predicted classes and their frequencies
    class_counts = Counter(predicted_classes)
    
    # Calculate the weighted frequency of each predicted class
    weighted_counts = {}
    for cls in class_counts:
        indices = [i for i, c in enumerate(predicted_classes) if c == cls]
        distances_to_cls = distances[indices]
        weights = 1 / (distances_to_cls + 1e-8)
        weighted_count = sum(weights)
        weighted_counts[cls] = weighted_count
    
    # Find the predicted class with the highest weighted frequency
    max_weighted_count = max(weighted_counts.values())
    if max_weighted_count >= threshold:
        predicted_class = max(weighted_counts, key=weighted_counts.get)
        return predicted_class
    else:
        return None

In [7]:
cam = cv2.VideoCapture(0)
while (cam.isOpened()):
    success, image = cam.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue
    else:
        cv2.imshow('imag', image)
        cv2.waitKey(1)
        faces = findFaces(image, modelD)
        for face in faces:
            cv2.imshow('face', face)
            cv2.waitKey(1)
            encoding = encodeFace(face, modelR)
            encoding = np.array(encoding).reshape(1, -1)
            neighDis, neighIndx = model.kneighbors(
                encoding, 6, return_distance=True)
            neighDis = neighDis[0]
            neighIndx = neighIndx[0]
            res = []
            for i, j in enumerate(neighIndx):
                neighDis[i] = neighDis[i]*10
                res.append(names[int(j)])

            print(knn_weighted_vote(neighDis, res,0.9))


[ WARN:0@114.180] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@114.181] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
